# Transport task

In [112]:
def gen_code(v, a, b):
    with open('script.py', 'w') as file:
        print('from cvxopt.modeling import variable, op', file=file)
        
    with open('script.py', 'a') as file:
        print(file=file)
        print(file=file)
        print('x = variable(%d, \'x\')' % (len(v) * len(v[0])), file=file)
        print(file=file)
        print('z = (', end='', file=file)
        
        k = 1
        for i, array in enumerate(v):
            if i != 0:
                print('     ', end='', file=file)
            for j, val in enumerate(array):
                end = '\t+\t'
                if j == len(array) - 1 and i == i == len(v) - 1:
                    end = '\t)'
                print('%d * x[%d]' % (val, i * k + j), end=end, file=file)
            print(file=file)
            k += 1
        print(file=file)
        
        k = 0
        for idx in range(len(v)):
            print('mass%d = (' % idx, end='', file=file)
            for i in range(k, k + len(v)):
                end= '\t+\t'
                if i == k + len(v) - 1:
                    end = '\t<= ' + str(a[idx]) + ')\n'
                print('x[%d]' % i, end=end, file=file)
            k += len(v)
        print(file=file)
        
        k = 0
        for idx in range(len(v), 2 * len(v)):
            print('mass%d = (' % idx, end='', file=file)
            for i in range(idx - len(v), len(v)**2, 5):
                end= '\t+\t'
                if i + 5 >= len(v) ** 2:
                    end = '\t== ' + str(b[idx - len(v)]) + ')\n'
                print('x[%d]' % (i), end=end, file=file)
        print(file=file)
        
        print('x_non_negative = (x>=0)', file=file)
        print(file=file)
        print('problem = op(z,[', end='', file=file)
        for idx in range(len(a) + len(b)):
            print('mass%d' % idx, end=', ', file=file)
        print('x_non_negative])', file=file)
        print(file=file)
        
        print('problem.solve(solver=\"glpk\")', file=file)
        print('problem.status', file=file)
        print('print("Result: {}".format(x.value))', file=file)
        print('print("Price: {}".format(problem.objective.value()[0]))', end='', file=file)

In [115]:
text = []
with open('t_task.txt') as f:
    for line in f:
        text.append(line.strip())

v = []
beta = []
alpha = [int(num) for num in text[0].split()]

for i in range(1, len(text)):
    line = [int(num) for num in text[i].split()]
    beta.append(line[0])
    v.append(line[1:])
    
gen_code(v, alpha, beta)

In [116]:
!python3 script.py

GLPK Simplex Optimizer, v4.65
35 rows, 25 columns, 75 non-zeros
      0: obj =   0.000000000e+00 inf =   8.000e+01 (5)
      7: obj =   2.300000000e+02 inf =   0.000e+00 (0)
*    25: obj =   9.000000000e+01 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Result: [ 1.00e+01]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 1.00e+01]
[ 0.00e+00]
[ 0.00e+00]
[ 1.50e+01]
[ 1.00e+01]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 2.00e+01]
[ 5.00e+00]
[ 0.00e+00]
[ 5.00e+00]
[ 0.00e+00]
[ 0.00e+00]
[ 5.00e+00]

Price: 90.0


# Game Task

In [147]:
def game(prices, P, Q):
    answer = {}
    l_price = max(min(price) for price in prices)
    u_price = min(max(price) for price in np.rot90(prices))
    
    if l_price == u_price:
        print("There is a saddle point! v = {%f}" % l_price)
        return answer

    H_pq = 0
    for strategy, p in zip(prices, P):
        H_pq += p * sum([price * q for price, q in zip(strategy, Q)])
    answer["H(P,Q)"] = H_pq
    
    for i, strategy in enumerate(np.rot90(prices), 1):
        answer["H(P,B{})".format(i)] = sum([price * p for price, p in zip(strategy, P)])
    
    return answer

In [148]:
import numpy as np


matrix = np.array([[ 8, 11,  9,  3,  5],
                   [ 3,  8,  3,  0,  6],
                   [ 9, 11,  4, 12,  6],
                   [11,  6, 11, 11,  1],
                   [ 8,  4,  0,  7,  7]])

P = [0.55, 0.0, 0.18,  0.0, 0.26]
Q = [ 0.0, 0.0, 0.18, 0.01,  0.8]

game(matrix, P, Q)

{'H(P,Q)': 5.5969,
 'H(P,B1)': 5.65,
 'H(P,B2)': 5.630000000000001,
 'H(P,B3)': 5.67,
 'H(P,B4)': 9.07,
 'H(P,B5)': 8.100000000000001}